In [ ]:
import pandas as pd
import dask.dataframe as dd
from dask import compute
pd.options.display.max_columns = None
import numpy as np
import dask
nan = np.nan
from dask.distributed import Client
client = dask.distributed.Client()

In [ ]:
path = 't_mdco_tfra400_Base3M_1.0.csv'

In [ ]:
%%time
df = pd.read_csv(path,
                 dtype={"producto_1_number":str,
                        "bin_2_number":str, 
                        "NSS":str,
                        "pasaporte":str,
                        "clave_id":str,
                        "clave_secundaria_text": str
                       })

In [ ]:
%%time
# Direccion
df['direccion'] = df.loc[:,'correo_id'].apply(lambda x: x[::-1])#,meta=('correo_id', 'object'))

In [ ]:
%%time
# Obtener nombre completo
def build_name(row):
    name = row['desc_text']
    lastShort = row['desc_id'][2:]
    index = name.find(lastShort.lower())
    return f'{name[:index].capitalize()} {name[index:].capitalize()}'
df['full_name'] = df[['desc_text', 'desc_id']].apply(build_name, axis=1)#, meta=('correo_id', 'object'))

In [ ]:
%%time
# Obtener email completo
def build_email(row):
    user = row['nombre_text']
    domain = str(row['apellid@_text']).replace('!','.com')
    return f'{user}@{domain}'
df['email'] = df[['nombre_text', 'apellid@_text']].apply(build_email, axis=1)#, meta=('correo_id', 'object'))

In [ ]:
%%time
# Obtener los 17 numeros del pasaporte
def build_passport(value):
    if str(value) == 'nan' or value == nan:
        return ''
    else:
        return str(int(str(value), 16)).zfill(17)
df['pasaporte_'] = df['pasaporte'].apply(
    build_passport,
    #meta=('pasaporte_', 'object')
)

In [ ]:
%%time
# Obtener los 17 numeros del pasaporte
def build_nss(value):
    if str(value) == 'nan' or value == nan:
        return ''
    else:
        return str(int(str(value), 8)).zfill(17)
df['NSS_'] = df['NSS'].apply(
    build_nss,
    #meta=('NSS_', 'object')
)

In [ ]:
%%time
# Obtener numeros de telefono
mapper = {
    'A' : '4',
    'B' : '8',
    'Z' : '2',
    'I' : '1',
    'S' : '5',
    'E' : '3',
    'l' : '1',
    'O' : '0'
}
def unleet(string):
    res = ''
    if str(string) == 'nan' or string == nan:
        return ''
    else:
        for c in string: res += mapper.get(c, c)
        return res
df['phone1'] = df['clave_secundaria_text'].apply(unleet)#, meta=('telefono_1', 'object'))
df['phone2'] = df['clave_primaria_text'].apply(unleet)#, meta=('telefono_2', 'object'))


In [ ]:
%%time
# Obtener fecha de algo
def get_date(row):
    clave = row['clave_id'][-2:]
    registro = row['registro.xls'][7:-4]
    return f'{registro[:4]}-{registro[4:]}-{clave}'
df['date0'] = df[['clave_id', 'registro.xls']].apply(get_date, axis=1)#, meta=('date0', 'object'))

In [ ]:
# CURP
def get_curp(row):
    pass

In [ ]:
%%time
res_df = df[['direccion', 'full_name', 'email', 'pasaporte_', 'NSS_', 'phone1', 'phone2', 'date0']]

In [ ]:
res_df.compute()

In [ ]:
# Dask sin cliente 4:46
# Pandas
# Dask con cliente
